In [1]:
import re
import copy
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
# 载入数据
ini_data = pd.read_csv("train.csv",low_memory=False)

In [2]:
# 填充数据
ini_data['program_type'] = ini_data['program_type'].fillna(method='bfill')
ini_data['trainee_engagement_rating'] = ini_data['trainee_engagement_rating'].fillna(method='ffill')
ini_data['is_handicapped'] = ini_data['is_handicapped'].fillna(method='bfill')
ini_data['education'] = ini_data['education'].fillna('High School Diploma')
ini_data['gender'] = ini_data['gender'].fillna(method='ffill')
ini_data['difficulty_level'] = ini_data['difficulty_level'].fillna(method='ffill')
ini_data['program_duration'] = ini_data['program_duration'].fillna(method='bfill')
ini_data['test_type'] = ini_data['test_type'].fillna(method='bfill')
ini_data['city_tier'] = ini_data['city_tier'].fillna(method='bfill')
# 因为这个值缺失不多 2占领了大多数，填充2
ini_data['total_programs_enrolled'] = ini_data['total_programs_enrolled'].fillna(2.0) 

"""
大体上硬编码
"""
ini_data.loc[ini_data['gender'] == 'M', "gender"] = 0
ini_data.loc[ini_data['gender'] == 'F', "gender"] = 1
# ini_data['gender']

ini_data.loc[ini_data['education'] == 'No Qualification', "education"] = 0
ini_data.loc[ini_data['education'] == 'High School Diploma', "education"] = 1
ini_data.loc[ini_data['education'] == 'Matriculation', "education"] = 2
ini_data.loc[ini_data['education'] == 'Bachelors', "education"] = 3
ini_data.loc[ini_data['education'] == 'Masters', "education"] = 4

ini_data.loc[ini_data['test_type'] == 'offline', "test_type"] = 0
ini_data.loc[ini_data['test_type'] == 'online', "test_type"] = 1
# ini_data['test_type']

ini_data.loc[ini_data['difficulty_level'] == 'easy', "difficulty_level"] = 0
ini_data.loc[ini_data['difficulty_level'] == 'intermediate', "difficulty_level"] = 1
ini_data.loc[ini_data['difficulty_level'] == 'hard', "difficulty_level"] = 2
ini_data.loc[ini_data['difficulty_level'] == 'vary hard', "difficulty_level"] = 3

ini_data.loc[ini_data['is_handicapped'] == 'N', "is_handicapped"] = 1
ini_data.loc[ini_data['is_handicapped'] == 'Y', "is_handicapped"] = 0

# 查看数据
# ini_data

,id_num,program_type,program_id,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,9389_150,Y,Y_1,136.0,150.0,0,1,9389.0,0,2,3.0,24.0,5.0,1,1.0,0
1,16523_44,T,T_1,131.0,44.0,0,0,16523.0,1,1,4.0,26.0,2.0,1,3.0,1
2,13987_178,Z,Z_2,120.0,178.0,1,0,13987.0,0,2,1.0,40.0,1.0,1,2.0,1
3,13158_32,T,T_2,117.0,32.0,0,0,13158.0,1,2,3.0,NaN,4.0,1,1.0,1
4,10591_84,V,V_3,131.0,84.0,0,1,10591.0,1,1,1.0,42.0,2.0,1,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,4379_115,Y,Y_4,121.0,115.0,0,2,4379.0,1,1,2.0,NaN,5.0,1,4.0,1
49994,3809_58,U,U_1,117.0,58.0,0,1,3809.0,1,1,3.0,30.0,2.0,1,3.0,1
49995,18109_42,T,T_2,117.0,42.0,1,0,18109.0,1,1,2.0,41.0,2.0,1,1.0,1
49996,6384_113,Y,Y_4,121.0,113.0,0,1,6384.0,0,2,3.0,NaN,2.0,1,5.0,1


In [3]:
# 深层拷贝，使得两者互不影响
droped_data = copy.deepcopy(ini_data)
# 删除空的(大多是年龄)以准备训练
droped_data = droped_data.dropna()
droped_data
# 完整的数据集用来预测
ini_data

,id_num,program_type,program_id,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,9389_150,Y,Y_1,136.0,150.0,0,1,9389.0,0,2,3.0,24.0,5.0,1,1.0,0
1,16523_44,T,T_1,131.0,44.0,0,0,16523.0,1,1,4.0,26.0,2.0,1,3.0,1
2,13987_178,Z,Z_2,120.0,178.0,1,0,13987.0,0,2,1.0,40.0,1.0,1,2.0,1
4,10591_84,V,V_3,131.0,84.0,0,1,10591.0,1,1,1.0,42.0,2.0,1,4.0,1
5,12531_23,T,T_3,134.0,23.0,0,1,12531.0,1,1,1.0,29.0,4.0,1,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49991,13936_61,U,U_1,134.0,61.0,1,0,13936.0,0,3,4.0,26.0,3.0,1,1.0,0
49992,12375_173,Z,Z_2,120.0,173.0,1,0,12375.0,1,2,4.0,26.0,1.0,1,1.0,1
49994,3809_58,U,U_1,117.0,58.0,0,1,3809.0,1,1,3.0,30.0,2.0,1,3.0,1
49995,18109_42,T,T_2,117.0,42.0,1,0,18109.0,1,1,2.0,41.0,2.0,1,1.0,1


,id_num,program_type,program_id,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,9389_150,Y,Y_1,136.0,150.0,0,1,9389.0,0,2,3.0,24.0,5.0,1,1.0,0
1,16523_44,T,T_1,131.0,44.0,0,0,16523.0,1,1,4.0,26.0,2.0,1,3.0,1
2,13987_178,Z,Z_2,120.0,178.0,1,0,13987.0,0,2,1.0,40.0,1.0,1,2.0,1
3,13158_32,T,T_2,117.0,32.0,0,0,13158.0,1,2,3.0,NaN,4.0,1,1.0,1
4,10591_84,V,V_3,131.0,84.0,0,1,10591.0,1,1,1.0,42.0,2.0,1,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,4379_115,Y,Y_4,121.0,115.0,0,2,4379.0,1,1,2.0,NaN,5.0,1,4.0,1
49994,3809_58,U,U_1,117.0,58.0,0,1,3809.0,1,1,3.0,30.0,2.0,1,3.0,1
49995,18109_42,T,T_2,117.0,42.0,1,0,18109.0,1,1,2.0,41.0,2.0,1,1.0,1
49996,6384_113,Y,Y_4,121.0,113.0,0,1,6384.0,0,2,3.0,NaN,2.0,1,5.0,1


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [5]:
# 初始化线性回归
lr = LinearRegression()

In [6]:
# 选用以下特征用来拟合
train_features = [ "trainee_engagement_rating","difficulty_level", "gender", "education", "city_tier",
                 "is_handicapped","test_type","is_pass"] 

In [7]:
# 本来是用k折交叉验证的，但又取消了
# k = KFold(n_splits=10, shuffle=True, random_state=1) 
# train_x = droped_data[train_features].iloc[0,:]

In [8]:
# 进行训练和拟合
save_predics = []
train_x = droped_data[train_features]
train_y = droped_data['age']
lr.fit(train_x,train_y)  # 对数据进行训练
test_predics = lr.predict(ini_data[train_features]) # 获取预测结果
save_predics.append(test_predics)  # 存放预测结果

LinearRegression()

In [9]:
# save_predics = np.concatenate(save_predics,axis=0)


In [33]:
# save_predics.max()
# save_predics.min()
save_predics = np.array(save_predics).T

In [34]:
# predicts_sample = []
# ini_data = ini_data.join(ini_data.pop('age'))
# ini_da
# len(ini_data['age'])
save_predics

array([[33.70593667],
       [36.42851272],
       [36.95841873],
       ...,
       [34.73471116],
       [39.42030254],
       [39.41197296]])

In [35]:
# 这里进行填充空缺填充，将拟合数据填充在原本空缺的地方
a = 0
for x,y in enumerate(ini_data['age']):
    """
    凭现在的水平只能逐个遍历然后赋值
    """
    a += 1
    if y != y and x%2 == 0:
        ini_data['age'][x] = save_predics[x]
#         ini_data['age'][x].fillna(save_predics[x])
print("Done")

<ipython-input-35-85405e168a8f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ini_data['age'][x] = save_predics[x]


Done


In [36]:
# ini_data['age'].isnull()
# 查看最大最小值，看拟合是否出问题
ini_data['age'].max()
ini_data['age'].min()
ini_data['age'].mean()

63.0

17.0

36.442806951792484

In [37]:
# 进行存储
ini_data.to_csv('waitfor_FE_filledage2.csv',encoding='utf-8')
print("Done")

Done


In [38]:
# ini_data.dropna()

,id_num,program_type,program_id,program_duration,test_id,test_type,difficulty_level,trainee_id,gender,education,city_tier,age,total_programs_enrolled,is_handicapped,trainee_engagement_rating,is_pass
0,9389_150,Y,Y_1,136.0,150.0,0,1,9389.0,0,2,3.0,24.000000,5.0,1,1.0,0
1,16523_44,T,T_1,131.0,44.0,0,0,16523.0,1,1,4.0,26.000000,2.0,1,3.0,1
2,13987_178,Z,Z_2,120.0,178.0,1,0,13987.0,0,2,1.0,40.000000,1.0,1,2.0,1
3,13158_32,T,T_2,117.0,32.0,0,0,13158.0,1,2,3.0,NaN,4.0,1,1.0,1
4,10591_84,V,V_3,131.0,84.0,0,1,10591.0,1,1,1.0,42.000000,2.0,1,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,4379_115,Y,Y_4,121.0,115.0,0,2,4379.0,1,1,2.0,NaN,5.0,1,4.0,1
49994,3809_58,U,U_1,117.0,58.0,0,1,3809.0,1,1,3.0,30.000000,2.0,1,3.0,1
49995,18109_42,T,T_2,117.0,42.0,1,0,18109.0,1,1,2.0,41.000000,2.0,1,1.0,1
49996,6384_113,Y,Y_4,121.0,113.0,0,1,6384.0,0,2,3.0,39.420303,2.0,1,5.0,1
